In [9]:
import numpy as np 

seed = 24 
rng = np.random.RandomState(seed)

In [10]:
from src.novelty.novelty import NoveltyFilter

V_az = np.load('/Users/ayushjain/Development/Research/alphazero/configs/novelty_V_az.npy')
V_human = np.load('/Users/ayushjain/Development/Research/alphazero/configs/novelty_V_human.npy')

filter = NoveltyFilter(V_human=V_human, V_az=V_az)

In [11]:
# load in the concept vectors from a specific runs directory 
runs_path = '/Users/ayushjain/Development/Research/alphazero/runs/concept_vectors/20260210_030953'

import os

# List all folders in the runs_path
layer_dirs = [d for d in os.listdir(runs_path) if os.path.isdir(os.path.join(runs_path, d))]

print("Layer directories:", layer_dirs)

# For each layer folder, list its subfolders (if any)
subfolders_dict = {}
for layer_dir in layer_dirs:
    layer_path = os.path.join(runs_path, layer_dir)
    subfolders = [d for d in os.listdir(layer_path) if os.path.isdir(os.path.join(layer_path, d))]
    subfolders_dict[layer_dir] = subfolders

print("Subfolders inside each layer directory:", subfolders_dict)


import os
import json
import numpy as np

output_path = os.path.join(runs_path, "novelty_results.json")

all_results = {}

def save_results(results_dict, path):
    tmp_path = path + ".tmp"
    with open(tmp_path, "w") as f:
        json.dump(results_dict, f, indent=2)
    os.replace(tmp_path, path)  # atomic write

# Iterate through all layer directories
for layer_dir in layer_dirs:

    concept_vector_path = os.path.join(
        runs_path,
        layer_dir,
        "layer=bn2",
        "concept_vector.npy"
    )

    if not os.path.exists(concept_vector_path):
        print(f"Missing: {concept_vector_path}")
        continue

    concept_vectors = np.load(concept_vector_path)

    filter_result = filter.check(concept_vectors)

    all_results[layer_dir] = {
        "novel": bool(filter_result[0]),
        "scores": np.asarray(filter_result[1]).tolist()
    }

    print(f"Processed {layer_dir} - shape {concept_vectors.shape}")

    # 🔥 SAVE AFTER EACH LAYER
    save_results(all_results, output_path)

# ✅ FINAL SAVE (in case loop never triggered or last update missed)
save_results(all_results, output_path)

print(f"Final results saved to {output_path}")

Layer directories: ['pos_0505', 'pos_2252', 'pos_3380', 'pos_2060', 'pos_0737', 'pos_2404', 'pos_1281', 'pos_0353', 'pos_0161', 'pos_2636', 'pos_1275', 'pos_1047', 'pos_0195', 'pos_1423', 'pos_3374', 'pos_2094', 'pos_3146', 'pos_1611', 'pos_2609', 'pos_0962', 'pos_0708', 'pos_2863', 'pos_3179', 'pos_2897', 'pos_2299', 'pos_1844', 'pos_0996', 'pos_1078', 'pos_0398', 'pos_2093', 'pos_3141', 'pos_1616', 'pos_1424', 'pos_3373', 'pos_1040', 'pos_0192', 'pos_1272', 'pos_1888', 'pos_0166', 'pos_2631', 'pos_2403', 'pos_1286', 'pos_0354', 'pos_2067', 'pos_0730', 'pos_0502', 'pos_2255', 'pos_3387', 'pos_1843', 'pos_0991', 'pos_1629', 'pos_2890', 'pos_2058', 'pos_2864', 'pos_0965', 'pos_0159', 'pos_0998', 'pos_1076', 'pos_0396', 'pos_1244', 'pos_1620', 'pos_3177', 'pos_2899', 'pos_3345', 'pos_2297', 'pos_1412', 'pos_0706', 'pos_3183', 'pos_2051', 'pos_2263', 'pos_0534', 'pos_2607', 'pos_0150', 'pos_1082', 'pos_0362', 'pos_2435', 'pos_3148', 'pos_1875', 'pos_1049', 'pos_0953', 'pos_1881', 'pos_263

In [14]:
any(res.get('novel', True) for res in all_results.values())

True

In [15]:
for res in all_results.values(): 
    if res.get('novel', True): 
        print(res)


{'novel': True, 'scores': [4.567109483689415e-05, 8.226115302904444e-06, 4.824138633457639e-06, 0.0003428118394045651, 0.00019245611795580368, 2.3961795020744844e-05, 0.00043138379333092745, 0.00026014470415471926, 1.57735158670617e-05, 1.8113188223833635e-05, 7.964576787321727e-07, 8.07905224670115e-19]}
{'novel': True, 'scores': [2.9107880850554393e-05, 2.7894034427660905e-05, 5.656016582821638e-06, 0.00034439306480457156, 0.0003341256647008458, 0.0009477396383823049, 0.00021888570070465602, 0.00030762303155123074, 0.00020729932946076486, 3.209859052279736e-05, 1.2326737371836384e-07, 4.5342153380375495e-20]}
{'novel': True, 'scores': [1.853152736514853e-05, 5.415837085173507e-05, 6.205018344121634e-05, 7.62961870230932e-06, 1.5161615998011795e-05, 3.368737565686078e-05, 0.00040162325649624117, 4.089900393658137e-05, 3.08655856559965e-05, 1.4635819098566756e-05, 2.6617469652924566e-07, 7.459089336466215e-19]}
{'novel': True, 'scores': [1.42732190842286e-05, 7.640863792098343e-06, 1.0